In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN, Bidirectional
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

traindata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_training-set.csv')
testdata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_testing-set.csv')

traindata=traindata.drop('id',axis=1)
testdata=testdata.drop('id',axis=1)

from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

traindata['proto'] = lb_make.fit_transform(traindata['proto'])
traindata['service'] = lb_make.fit_transform(traindata['service'])
traindata['state'] = lb_make.fit_transform(traindata['state'])

testdata['proto'] = lb_make.fit_transform(testdata['proto'])
testdata['service'] = lb_make.fit_transform(testdata['service'])
testdata['state'] = lb_make.fit_transform(testdata['state'])

X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,-1]
x = testdata.iloc[:,1:42]
y = testdata.iloc[:,-1]

Using TensorFlow backend.


In [2]:
Arr = X.values
arr = x.values

In [3]:
import librosa
Arr_Delta = librosa.feature.delta(Arr)
arr_delta = librosa.feature.delta(arr)

In [4]:
Delta1 = pd.DataFrame(Arr_Delta)
delta1 = pd.DataFrame(arr_delta)

In [5]:
scaler = Normalizer().fit(Delta1)
Delta1 = scaler.transform(Delta1)
scaler = Normalizer().fit(delta1)
delta1 = scaler.transform(delta1)

Y = np.array(Y)
y = np.array(y)

Delta1 = np.reshape(Delta1, (Delta1.shape[0],Delta1.shape[1],1))
delta1 = np.reshape(delta1, (delta1.shape[0],delta1.shape[1],1))


scaler = Normalizer().fit(X)
X = scaler.transform(X)
scaler = Normalizer().fit(x)
x = scaler.transform(x)

Y = np.array(Y)
y = np.array(y)

X = np.reshape(X, (X.shape[0],X.shape[1],1))
x = np.reshape(x, (x.shape[0],x.shape[1],1))


In [6]:
delta1.shape

(82332, 41, 1)

In [7]:
from keras.layers import Input, Convolution1D, Dense, Dropout, Flatten, MaxPooling1D, concatenate, Reshape
from keras.models import Model

lstm_output_size=70
Xshape = Input(shape=(41, 1))
Delta1shape = Input(shape=(41, 1))

x1 = Convolution1D(64, 3, border_mode="same",activation="relu")(Xshape)
x1 = MaxPooling1D(pool_length=(2))(x1)
x1 = Dropout(0.5)(x1)
   
x2 = Convolution1D(64, 3, border_mode="same",activation="relu")(Delta1shape)
x2 = MaxPooling1D(pool_length=(2))(x2)
x2 = Dropout(0.5)(x2)

output = concatenate([x1, x2])

x3 = Bidirectional(LSTM(lstm_output_size))(output)
x3 = Dropout(0.1)(x3)
x3 = Dense(1, activation="sigmoid")(x3)

model = Model([Xshape, Delta1shape],x3)
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
print(model.summary())
model.fit([X,Delta1], Y, epochs=200, validation_data=([x,delta1], y))


/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
  
/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  if __name__ == '__main__':
/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
  if sys.path[0] == '':
/home/ritika/.virtualenvs/project/lib/python3.5/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  del sys.path[0]


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 41, 1)        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 41, 1)        0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 41, 64)       256         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 41, 64)       256         input_2[0][0]                    
____________________________________________________________________________________________

175341/175341 [==============================] - 168s 956us/step - loss: 0.2670 - accuracy: 0.8761 - val_loss: 0.4528 - val_accuracy: 0.7418
Epoch 89/200
175341/175341 [==============================] - 168s 961us/step - loss: 0.2666 - accuracy: 0.8767 - val_loss: 0.4741 - val_accuracy: 0.7333
Epoch 90/200
175341/175341 [==============================] - 169s 963us/step - loss: 0.2671 - accuracy: 0.8767 - val_loss: 0.4446 - val_accuracy: 0.7431
Epoch 91/200
175341/175341 [==============================] - 170s 968us/step - loss: 0.2663 - accuracy: 0.8768 - val_loss: 0.4570 - val_accuracy: 0.7549
Epoch 92/200
175341/175341 [==============================] - 170s 967us/step - loss: 0.2670 - accuracy: 0.8760 - val_loss: 0.4310 - val_accuracy: 0.7358
Epoch 93/200
175341/175341 [==============================] - 170s 969us/step - loss: 0.2666 - accuracy: 0.8765 - val_loss: 0.4447 - val_accuracy: 0.7544
Epoch 94/200
175341/175341 [==============================] - 170s 968us/step - loss: 0.2

Epoch 194/200
175341/175341 [==============================] - 163s 932us/step - loss: 0.2572 - accuracy: 0.8806 - val_loss: 0.4474 - val_accuracy: 0.7375
Epoch 195/200
175341/175341 [==============================] - 163s 932us/step - loss: 0.2578 - accuracy: 0.8802 - val_loss: 0.4229 - val_accuracy: 0.7447
Epoch 196/200
175341/175341 [==============================] - 164s 936us/step - loss: 0.2533 - accuracy: 0.8828 - val_loss: 0.4360 - val_accuracy: 0.7457
Epoch 197/200
175341/175341 [==============================] - 165s 943us/step - loss: 0.2560 - accuracy: 0.8812 - val_loss: 0.4433 - val_accuracy: 0.7493
Epoch 198/200
175341/175341 [==============================] - 164s 936us/step - loss: 0.2556 - accuracy: 0.8814 - val_loss: 0.4868 - val_accuracy: 0.7384
Epoch 199/200
175341/175341 [==============================] - 165s 938us/step - loss: 0.2556 - accuracy: 0.8812 - val_loss: 0.4781 - val_accuracy: 0.7472
Epoch 200/200
175341/175341 [==============================] - 164s 93

Epoch 200/200
175341/175341 [==============================] - 164s 937us/step - loss: 0.2555 - accuracy: 0.8807 - val_loss: 0.4269 - val_accuracy: 0.7563